# QCNN Classification for Malware Detection

---
---

## Step-1: Import Dependencies and Setup
---

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

## Step-2: QCNN Circuit Definition

---

In [ ]:
@qml.qnode(dev, interface="torch")
def qcnn_circuit(x, weights):
    # x shape: (5, 2, 15)
    # Embed each section
    for s in range(5):
        angular_hybrid_4_block(x[s, 0], wires=[0,1,2,3])
        angular_hybrid_4_block(x[s, 1], wires=[4,5,6,7])
    # Parameterized convolution+pooling layers
    for layer in weights:
        for w in range(0, n_qubits, 2):
            qml.templates.StronglyEntanglingLayers(layer, wires=[w, w+1])
        # simple pooling: measure and reset second qubit
        for w in range(1, n_qubits, 2):
            qml.PauliZ(wires=w)
            qml.PauliX(wires=w)
            qml.PauliX(wires=w)
    return qml.expval(qml.PauliZ(0))